In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from numpy.linalg import inv
import umap
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go

In [ ]:
bio = pd.read_csv('/Users/ingridreyes/Desktop/Tesis/embeddings_10000_biobert.csv')
prot = pd.read_csv('/Users/ingridreyes/Desktop/Tesis/embeddings_10000_prot_seq.csv')

In [ ]:
#eliminaer la columa de las secuencias
bio_numeric = bio.select_dtypes(include=[np.number])
prot_numeric = prot.select_dtypes(include=[np.number])

In [ ]:
#concatenar los datos de biobert y protbert
X = np.concatenate((bio_numeric.values, prot_numeric.values), axis=1)

In [ ]:
#matriz de covarianza de maha 
#covarianza de la matriz x 
#owvar=False Cuando las observaciones están en filas, true cuando estan en columnas
cov = np.cov(X, rowvar=False)
lambda_reg = 0.01
#regularización de Tikhonov o ridge regularization, aumenta ligeramente la varianza de cada variable, evitando que el determinante sea 0.
#evita singularidad, una matriz que no se puede invertir.
cov_reg = cov + lambda_reg * np.eye(cov.shape[0])
#cov.shape[0] el número de filas (y columnas) de la matriz de covarianza
#np.eye(cov.shape[0]) crea una matriz identidad del mismo tamaño que tu matriz de covarianza.
#lambda_reg * np.eye(...) multiplica esa identidad por λ 
inv_cov = inv(cov_reg)

In [ ]:
#distancias de maha 
#Aqui solo recorre la mitad superior, porque es lo mismo para lo de abajo (:
N = X.shape[0]
D = np.zeros((N, N))

for i in range(N):
    for j in range(i+1, N):
        d = mahalanobis(X[i], X[j], inv_cov)
        #saca la distancia de la proteina i y la j y la inversa de la matriz de covarianza
        D[i, j] = d
        D[j, i] = d

Nota (: 
n_neighbors grandes captura los datos de forma más global
min_dits pequeños 

In [ ]:
#Hacer la reducción de UMAP
reducer = umap.UMAP(
    n_neighbors=30,  # ajusta para familias genéticas (más grande = más global)
    min_dist=0.1, #este parámeyro entre más pequeño sea los clústeres quedan más definidos.
    n_components=3, #gráfico 3D
    metric='precomputed',  # usamos la matriz de distancias
    random_state=42
)
embedding = reducer.fit_transform(D) #D[i, j] es la distancia entre la proteína i y la proteína j, que se calculo arriba (:

eps=0.5 el radio de vecindad: define qué tan cerca deben estar dos puntos para ser considerados del mismo grupo.
min_samples=10 número mínimo de puntos dentro de ese radio para formar un cluster.
metric='euclidean'usa la distancia euclidiana 
.fit(embedding) → aplica DBSCAN sobre las coordenadas reducidas por UMAP 

In [ ]:
db = DBSCAN(eps=0.5, min_samples=10, metric='euclidean').fit(embedding)
labels = db.labels_

go.Figure() crea una figura de Plotly.
go.Scatter3d() dibuja un gráfico de dispersión 3D.
x, y, z las coordenadas UMAP 
mode='markers' en puntos (:
marker cómo se ven los puntos: tañmalo, color, etc.
text=descriptions :descripción de cada proteína al pasar el cursor.
hovertemplate :personaliza lo que aparece en el recuadro flotante al pasar el mouse.

In [ ]:
desc_df = pd.read_csv('/Users/ingridreyes/Desktop/Tesis/mahalanobis_cosine.csv')
descriptions = desc_df.iloc[:, 0].values #para tomar las descripciones que estan en la primera columna del archivo

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=embedding[:, 0],
            y=embedding[:, 1],
            z=embedding[:, 2],
            mode='markers',
            marker=dict(
                size=4,
                color=labels,
                colorscale='Viridis',
                opacity=0.8
            ),
            text=descriptions,
            hovertemplate="<b>%{text}</b><extra></extra>"
        )
    ]
)

fig.update_layout(
    title="UMAP + DBSCAN (Distancia de Mahalanobis)",
    scene=dict(xaxis_title='UMAP1', yaxis_title='UMAP2', zaxis_title='UMAP3'),
    height=700
)

fig.show()